In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pprint as pp
# import sys
# !{sys.executable} -m pip install sklearn

In [2]:
def calculate_entropy(df_label):
    classes, class_counts = np.unique(df_label, return_counts= True)
   # print(np.sum(class_counts))
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))* np.log2(class_counts[i]/np.sum(class_counts))
                           for i in range(len(classes))])
    
    return entropy_value



In [3]:
def calculate_info_gain(X_train,label,feature):
    entropy = calculate_entropy(X_train[label])
    #print(entropy)
    values,feature_counts = np.unique(X_train[feature], return_counts = True)
    info_gain = entropy - (np.sum([(feature_counts[i]/np.sum(feature_counts))*
                               calculate_entropy(X_train.where(X_train[feature]== values[i]).dropna()[label])
                               for i in range(len(values))]))
    return info_gain
    


In [4]:
def build_tree(df_train,df,label,features):
    u_class = np.unique(df_train[label])
    if(len(u_class) <= 1):
        return u_class[0]
    elif len(features) == 0:
        return -1
    else :
        gains = [calculate_info_gain(df_train,label,feature)
                for feature in features]
        min_feature = features[np.argmax(gains)]
       # print(min_feature)
        d_tree = {min_feature:{}}
        features.remove(min_feature)
       # print(features)
        
        for  val in np.unique(df_train[min_feature]):
            min_val = df_train.where(df_train[min_feature]==val).dropna()
            min_tree = build_tree(min_val,df,label,features)
            
            d_tree[min_feature][val] = min_tree
    return d_tree
        

In [5]:
def test(df_test, dtree):
    #print(type(dtree))
    for nodes in dtree.keys():
        val = 0
        dtree = dtree[nodes][df_test[nodes]]
        if type(dtree) is dict:
            val = test(df_test,dtree)
        else : 
            val = dtree
            break;
    return val
            

In [6]:
def calculate_accuracy(df_test, dtree):
    accuracy = 0
    for i in range(len(df_test)):
        df_test_copy = df_test.iloc[0,:]
        got = int(test(df_test_copy,dtree))
        #print(type(got))
        if int(df_test_copy["Class"]) == got:
            accuracy+=1
    return (accuracy/len(df_test))*100
        
    

In [7]:
df = pd.read_csv("wine.data")
df.dropna(inplace=True)
col=["Class","Alchohol","Malic Acid","Ash","Alcalinity","Magnesium","Phenols","Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color","Hue","Diluted wines","Proline"]
df.columns = col;
labels = ['Low','Medium', 'High']


#print(len(df))
for x in col[1:] :
    df[x]= pd.cut(df[x],bins=3,labels=labels)

# X = df.iloc[:,1:]
# y = df.iloc[:,0]
df_train,df_test=train_test_split(df,test_size=0.2,random_state=42)
#print(df_test.head(2))
#print(X['Alchohol'])
#print(X_train.head,y_train.head)
#calculate_info_gain(df_train,"Class","Alchohol")
#print(calculate_entropy(y_train))
parent = None
decision_tree=build_tree(df_train,df_train,"Class",col[1:]);
print("THE DECISION TREE")
pp.pprint(decision_tree)
print("\n\nAccuracy is ",calculate_accuracy(df_test,decision_tree),"%")




THE DECISION TREE
{'Proline': {'High': 1.0,
             'Low': {'Color': {'High': 3.0,
                               'Low': 2.0,
                               'Medium': {'Flavanoids': {'High': 2.0,
                                                         'Low': {'Alcalinity': {'High': 3.0,
                                                                                'Low': 2.0,
                                                                                'Medium': 3.0}},
                                                         'Medium': {'Malic Acid': {'Low': 2.0,
                                                                                   'Medium': 1.0}}}}}},
             'Medium': {'Phenols': {'High': {'Magnesium': {'High': 2.0,
                                                           'Low': 1.0,
                                                           'Medium': 1.0}},
                                    'Low': {'Ash': {'Low': 2.0, 'Medium': 3.0}},
                  